In [1]:
using CSV
using DataFrames
using Random: MersenneTwister
using StatsBase: mean, std

include("../src/Simulation.jl")
using .Simulation: Param, Model, run!, Strategy, C

println("Julia $(VERSION)")
println("Thread count: $(Threads.nthreads())")

Julia 1.10.2
Thread count: 12


# Regional Variability

In [3]:
function area_C_rate(N::Int, strategy_mat::Matrix{Strategy}, t::Int, span::Int, rich_flag::Bool)::Float64
    # prime node or the poorest node
    node = rich_flag ? peak_node_vec[t] : ((peak_node_vec[t] + Int(N / 2)) % N)

    strategy_vec = if node + span > N
        [strategy_mat[t, (node - span):N]; strategy_mat[t, 1:(node + span) % N]]
    elseif node - span < 1
        [strategy_mat[t, (node - span + N):N]; strategy_mat[t, 1:(node + span)]]
    else
        strategy_mat[t, (node - span):(node + span)]
    end

    return mean(strategy_vec .== C)
end

function moving_average(data::Vector{<:Real}, window_size::Int = 10)::Vector{Float64}
    return [mean(data[max(1, i-window_size):min(end, i+window_size)]) for i in 1:length(data)]
end;

In [4]:
C_rate₀ = 0.0
b = 1.8
relationship_increment_factor = 1.0
resource_decrement_factor = 0.02
resource_limit_μ = 0.5
resource_limit_β = 0.0
resource_limit_σ = 0.0
peak_node_variability = 16
random_seed = 3

p = Param(
    N = 100, k₀ = 4, generations = 100_000,
    b = b,
    C_rate₀ = C_rate₀,
    peak_node_variability = peak_node_variability,
    relationship_increment_factor = relationship_increment_factor,
    resource_decrement_factor = resource_decrement_factor,
    resource_limit_μ = resource_limit_μ,
    resource_limit_β = resource_limit_β,
    resource_limit_σ = resource_limit_σ,
)
m = Model(p, MersenneTwister(random_seed))
C_rate_vec, _, _, peak_node_vec, strategy_mat, degree_mat, _, _, average_C_neighbour_rate_vec = run!(m, MersenneTwister(random_seed));

In [5]:
df = DataFrame()

time_span = 100
keep = p.generations - 1_000
max_t = length(C_rate_vec)
window = 10

# Temporary variables
## C rate
rich_area_C_rate_vec = [area_C_rate(p.N, strategy_mat, t, 12, true) for t in 1:p.generations]
poor_area_C_rate_vec = [area_C_rate(p.N, strategy_mat, t, 12, false) for t in 1:p.generations];
moving_C_rate_vec = moving_average(C_rate_vec, window)
moving_rich_area_C_rate_vec = moving_average(rich_area_C_rate_vec, window)
moving_poor_area_C_rate_vec = moving_average(poor_area_C_rate_vec, window)
## Network
std_degree_vec = vec(std(degree_mat, dims=2))
nodes_above_8_vec = [sum(degree_mat[x, :] .>= 8) for x in 1:max_t]
nodes_above_10_vec = [sum(degree_mat[x, :] .>= 10) for x in 1:max_t]
moving_std_degree_vec = moving_average(std_degree_vec, window)
moving_nodes_above_8_vec = moving_average(nodes_above_8_vec, window)
moving_nodes_above_10_vec = moving_average(nodes_above_10_vec, window)
## Environmental variablity
prime_node_variation_vec = [peak_node_vec[x] - peak_node_vec[x-1] for x in 2:max_t]
prime_node_variation_vec = [x > 50 ? x - 100 : x for x in prime_node_variation_vec]
prime_node_variation_vec = [x < -50 ? x + 100 : x for x in prime_node_variation_vec]

# Object variables
## C rate
df.moving_C_rate_var = moving_C_rate_vec[(max_t - keep):max_t] .- moving_C_rate_vec[(max_t - keep - time_span):(max_t - time_span)]
df.moving_rich_area_C_rate_var = moving_C_rate_vec[(max_t - keep):max_t] .- moving_C_rate_vec[(max_t - keep - time_span):(max_t - time_span)]
df.moving_poor_area_C_rate_var = moving_C_rate_vec[(max_t - keep):max_t] .- moving_C_rate_vec[(max_t - keep - time_span):(max_t - time_span)]
## Network
df.moving_std_degree_var = moving_std_degree_vec[(max_t - keep):max_t] .- moving_std_degree_vec[(max_t - keep - time_span):(max_t - time_span)]
df.moving_nodes_above_8_var = moving_nodes_above_8_vec[(max_t - keep):max_t] .- moving_nodes_above_8_vec[(max_t - keep - time_span):(max_t - time_span)]
df.moving_nodes_above_10_var = moving_nodes_above_10_vec[(max_t - keep):max_t] .- moving_nodes_above_10_vec[(max_t - keep - time_span):(max_t - time_span)]

# Features
## x generation prior C rate
df.prior_C_rate = C_rate_vec[(end - keep - time_span):(end - time_span)]
df.prior_rich_C_rate = rich_area_C_rate_vec[(end - keep - time_span):(end - time_span)]
df.prior_poor_C_rate = poor_area_C_rate_vec[(end - keep - time_span):(end - time_span)]
## x generation mean C rate
df.mean_C_rate = [mean(C_rate_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
df.mean_rich_C_rate = [mean(rich_area_C_rate_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
df.mean_poor_C_rate = [mean(poor_area_C_rate_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
## Network
### degree std
df.mean_std_degree = [mean(std_degree_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
### count of hub
df.mean_hub8_count = [mean(nodes_above_8_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
df.mean_hub10_count = [mean(nodes_above_10_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
## Environmental variablity
## Mean variation (移動距離の絶対値の平均: 1行って1戻ったら平均1)
df.pn_mean_var = [mean(abs.(prime_node_variation_vec[(x-time_span+1):x-1])) for x in time_span:length(prime_node_variation_vec)][end - keep:end]
## Sum variation (総移動距離: 1行って1戻ったら合計0)
df.pn_sum_var = [abs(sum(prime_node_variation_vec[(x-time_span+1):x-1])) for x in time_span:length(prime_node_variation_vec)][end - keep:end]
## Max variation (最大移動距離: 1行って2戻ったら最大2)
df.pn_max_var = Float64.([maximum(abs.(prime_node_variation_vec[(x-time_span+1):x-1])) for x in time_span:length(prime_node_variation_vec)])[end - keep:end]
## Mixed variables
### mean_C_rate
df.param_mean1 = (1 .- df.mean_C_rate) .* df.pn_sum_var
df.param_mean2 = (1 .- df.mean_rich_C_rate) .* df.pn_sum_var
df.param_mean3 = (1 .- df.mean_poor_C_rate) .* df.pn_sum_var
df.param_mean4 = (1 .- df.mean_C_rate) .* df.pn_mean_var
df.param_mean5 = (1 .- df.mean_rich_C_rate) .* df.pn_mean_var
df.param_mean6 = (1 .- df.mean_poor_C_rate) .* df.pn_mean_var
### prior_c_rate
df.param_prior1 = (1 .- df.prior_C_rate) .* df.pn_sum_var
df.param_prior2 = (1 .- df.prior_rich_C_rate) .* df.pn_sum_var
df.param_prior3 = (1 .- df.prior_poor_C_rate) .* df.pn_sum_var
df.param_prior4 = (1 .- df.prior_C_rate) .* df.pn_mean_var
df.param_prior5 = (1 .- df.prior_rich_C_rate) .* df.pn_mean_var
df.param_prior6 = (1 .- df.prior_poor_C_rate) .* df.pn_mean_var

CSV.write("EDA_Regional_$(time_span).csv", df)
df

Row,moving_C_rate_var,moving_rich_area_C_rate_var,moving_poor_area_C_rate_var,moving_std_degree_var,moving_nodes_above_8_var,moving_nodes_above_10_var,prior_C_rate,prior_rich_C_rate,prior_poor_C_rate,mean_C_rate,mean_rich_C_rate,mean_poor_C_rate,mean_std_degree,mean_hub8_count,mean_hub10_count,pn_mean_var,pn_sum_var,pn_max_var,param_mean1,param_mean2,param_mean3,param_mean4,param_mean5,param_mean6,param_prior1,param_prior2,param_prior3,param_prior4,param_prior5,param_prior6
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.116667,0.116667,0.116667,0.100244,1.2381,0.428571,0.04,0.04,0.04,0.107525,0.10099,0.110099,2.10801,7.33663,1.79208,8.05051,89,16.0,79.4303,80.0119,79.2012,7.18488,7.23748,7.16415,85.44,85.44,85.44,7.72848,7.72848,7.72848
2,0.114762,0.114762,0.114762,0.121684,1.38095,0.52381,0.02,0.04,0.0,0.10901,0.102178,0.111683,2.10408,7.34653,1.75248,8.14141,74,16.0,65.9333,66.4388,65.7354,7.25392,7.30954,7.23216,72.52,71.04,74.0,7.97859,7.81576,8.14141
3,0.112381,0.112381,0.112381,0.100755,1.42857,0.285714,0.03,0.04,0.0,0.110594,0.103762,0.113663,2.10733,7.36634,1.77228,8.10101,70,16.0,62.2584,62.7366,62.0436,7.20509,7.26043,7.18022,67.9,67.2,70.0,7.85798,7.77697,8.10101
4,0.11,0.11,0.11,0.0668995,1.2381,-0.0952381,0.04,0.04,0.04,0.112277,0.106139,0.115644,2.10953,7.36634,1.78218,8.20202,80,16.0,71.0178,71.5089,70.7485,7.28112,7.33147,7.25351,76.8,76.8,76.8,7.87394,7.87394,7.87394
5,0.108095,0.108095,0.108095,0.0504758,0.952381,-0.047619,0.03,0.04,0.0,0.113861,0.107723,0.117624,2.11124,7.42574,1.76238,8.19192,79,16.0,70.005,70.4899,69.7077,7.25918,7.30946,7.22835,76.63,75.84,79.0,7.94616,7.86424,8.19192
6,0.105714,0.105714,0.105714,0.0383669,0.809524,-0.190476,0.02,0.04,0.0,0.115347,0.109307,0.118812,2.10733,7.40594,1.73267,8.23232,75,16.0,66.349,66.802,66.0891,7.28275,7.33247,7.25423,73.5,72.0,75.0,8.06768,7.90303,8.23232
7,0.100476,0.100476,0.100476,0.0429351,1.04762,-0.190476,0.03,0.04,0.0,0.117129,0.110099,0.12,2.10959,7.40594,1.74257,8.27273,71,16.0,62.6839,63.183,62.48,7.30375,7.36191,7.28,68.87,68.16,71.0,8.02455,7.94182,8.27273
8,0.0952381,0.0952381,0.0952381,0.0152911,0.857143,-0.428571,0.05,0.04,0.04,0.118812,0.111683,0.121584,2.11136,7.42574,1.75248,8.23232,43,16.0,37.8911,38.1976,37.7719,7.25423,7.31291,7.2314,40.85,41.28,41.28,7.82071,7.90303,7.90303
9,0.0895238,0.0895238,0.0895238,0.00155149,0.619048,-0.380952,0.04,0.04,0.08,0.120099,0.112475,0.122772,2.11438,7.46535,1.76238,8.33333,33,16.0,29.0367,29.2883,28.9485,7.33251,7.39604,7.31023,31.68,31.68,30.36,8.0,8.0,7.66667


# Universal Variability

In [7]:
C_rate₀ = 0.0
b = 1.8
relationship_increment_factor = 1.0
resource_decrement_factor = 0.02
resource_limit_μ = 0.5
resource_limit_β = 0.9
resource_limit_σ = 0.1
peak_node_variability = 0
random_seed = 2

p = Param(
    N = 100, k₀ = 4, generations = 10_000,
    b = b,
    C_rate₀ = C_rate₀,
    peak_node_variability = peak_node_variability,
    relationship_increment_factor = relationship_increment_factor,
    resource_decrement_factor = resource_decrement_factor,
    resource_limit_μ = resource_limit_μ,
    resource_limit_β = resource_limit_β,
    resource_limit_σ = resource_limit_σ,
)
m = Model(p, MersenneTwister(random_seed))
C_rate_vec, average_resource_vec, death_count_vec, peak_node_vec, strategy_mat, degree_mat, average_distance_vec, clustering_coefficient_vec, average_C_neighbour_rate_vec = run!(m, MersenneTwister(random_seed));

In [8]:
df = DataFrame()

time_span = 100
keep = p.generations - 1_000
max_t = length(C_rate_vec)
window = 10

# Temporary variables
## C rate
rich_area_C_rate_vec = [area_C_rate(p.N, strategy_mat, t, 12, true) for t in 1:p.generations]
poor_area_C_rate_vec = [area_C_rate(p.N, strategy_mat, t, 12, false) for t in 1:p.generations];
moving_C_rate_vec = moving_average(C_rate_vec, window)
moving_rich_area_C_rate_vec = moving_average(rich_area_C_rate_vec, window)
moving_poor_area_C_rate_vec = moving_average(poor_area_C_rate_vec, window)
## Network
std_degree_vec = vec(std(degree_mat, dims=2))
nodes_above_8_vec = [sum(degree_mat[x, :] .>= 8) for x in 1:max_t]
nodes_above_10_vec = [sum(degree_mat[x, :] .>= 10) for x in 1:max_t]
moving_std_degree_vec = moving_average(std_degree_vec, window)
moving_nodes_above_8_vec = moving_average(nodes_above_8_vec, window)
moving_nodes_above_10_vec = moving_average(nodes_above_10_vec, window)
## Environmental variablity
resource_limit_var_vec = abs.(m.resource_limit_vec[2:end] .- m.resource_limit_vec[1:(end-1)])

# Object variables
## C rate
df.moving_C_rate_var = moving_C_rate_vec[(max_t - keep):max_t] .- moving_C_rate_vec[(max_t - keep - time_span):(max_t - time_span)]
df.moving_rich_area_C_rate_var = moving_C_rate_vec[(max_t - keep):max_t] .- moving_C_rate_vec[(max_t - keep - time_span):(max_t - time_span)]
df.moving_poor_area_C_rate_var = moving_C_rate_vec[(max_t - keep):max_t] .- moving_C_rate_vec[(max_t - keep - time_span):(max_t - time_span)]
## Network
df.moving_std_degree_var = moving_std_degree_vec[(max_t - keep):max_t] .- moving_std_degree_vec[(max_t - keep - time_span):(max_t - time_span)]
df.moving_nodes_above_8_var = moving_nodes_above_8_vec[(max_t - keep):max_t] .- moving_nodes_above_8_vec[(max_t - keep - time_span):(max_t - time_span)]
df.moving_nodes_above_10_var = moving_nodes_above_10_vec[(max_t - keep):max_t] .- moving_nodes_above_10_vec[(max_t - keep - time_span):(max_t - time_span)]

# Features
## x generation prior C rate
df.prior_C_rate = C_rate_vec[(end - keep - time_span):(end - time_span)]
df.prior_rich_C_rate = rich_area_C_rate_vec[(end - keep - time_span):(end - time_span)]
df.prior_poor_C_rate = poor_area_C_rate_vec[(end - keep - time_span):(end - time_span)]
## x generation mean C rate
df.mean_C_rate = [mean(C_rate_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
df.mean_rich_C_rate = [mean(rich_area_C_rate_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
df.mean_poor_C_rate = [mean(poor_area_C_rate_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
## Network
### degree std
df.mean_std_degree = [mean(std_degree_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
### count of hub
df.mean_hub8_count = [mean(nodes_above_8_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
df.mean_hub10_count = [mean(nodes_above_10_vec[(x - time_span):x]) for x in (max_t - keep):max_t]
## Environmental variablity
df.resource_limit_var = moving_average(resource_limit_var_vec, 10)[(end - keep):end]
df.mean_resource_limit = [mean(m.resource_limit_vec[(x - time_span):x]) for x in (max_t - keep):max_t]

CSV.write("EDA_Universal_$(time_span).csv", df)
df

Row,moving_C_rate_var,moving_rich_area_C_rate_var,moving_poor_area_C_rate_var,moving_std_degree_var,moving_nodes_above_8_var,moving_nodes_above_10_var,prior_C_rate,prior_rich_C_rate,prior_poor_C_rate,mean_C_rate,mean_rich_C_rate,mean_poor_C_rate,mean_std_degree,mean_hub8_count,mean_hub10_count,resource_limit_var,mean_resource_limit
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.0842857,-0.0842857,-0.0842857,0.176477,2.7619,0.285714,0.16,0.12,0.12,0.159802,0.115644,0.167129,1.97585,5.9802,1.0495,0.0701668,0.592901
2,-0.0904762,-0.0904762,-0.0904762,0.176677,2.85714,0.380952,0.16,0.12,0.08,0.159406,0.115248,0.167525,1.9759,5.9901,1.0495,0.0649288,0.591675
3,-0.0933333,-0.0933333,-0.0933333,0.177106,3.0,0.47619,0.18,0.16,0.16,0.158812,0.114851,0.167129,1.97303,6.0099,1.0297,0.0703355,0.588476
4,-0.0971429,-0.0971429,-0.0971429,0.177664,2.95238,0.571429,0.21,0.16,0.2,0.158218,0.114059,0.166733,1.97494,6.05941,1.0396,0.0710288,0.584273
5,-0.101905,-0.101905,-0.101905,0.14152,2.71429,0.571429,0.23,0.16,0.2,0.157228,0.113267,0.166337,1.97546,6.06931,1.0297,0.0672767,0.581991
6,-0.106667,-0.106667,-0.106667,0.131388,2.61905,0.619048,0.27,0.16,0.28,0.156238,0.112475,0.165941,1.97587,6.08911,1.0396,0.0688128,0.581118
7,-0.109048,-0.109048,-0.109048,0.114673,2.28571,0.666667,0.3,0.16,0.32,0.154653,0.111683,0.164356,1.97879,6.14851,1.07921,0.0727371,0.578755
8,-0.11619,-0.11619,-0.11619,0.118651,2.2381,0.714286,0.29,0.16,0.32,0.152871,0.110891,0.162772,1.97897,6.17822,1.07921,0.0736557,0.576266
9,-0.124762,-0.124762,-0.124762,0.141108,2.14286,0.952381,0.29,0.16,0.36,0.151089,0.110099,0.160396,1.97776,6.17822,1.06931,0.0809304,0.576063
